In [1]:
import os
print(os.listdir("../input"))

import numpy as np
import pandas as pd

import time
from tqdm import tqdm
tqdm.pandas()
import gc

import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = "../input"

['train.csv', 'embeddings', 'test.csv', 'train.csv.zip', 'sample_submission.csv', 'embeddings.zip', 'test.csv.zip', 'sample_submission.csv.zip']


In [2]:
train_df = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_PATH, "test.csv"))
print("Train shape : ", train_df.shape)
print("Test shape : ", test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [3]:
insincere_df = train_df.loc[train_df["target"] == 1]
sincere_df = train_df.loc[train_df["target"] == 0]
pd.set_option("max_colwidth",200)
print(insincere_df.head(10)["question_text"])
print(sincere_df.head(10)["question_text"])

22                                                                                                      Has the United States become the largest dictatorship in the world?
30                                                                                   Which babies are more sweeter to their parents? Dark skin babies or light skin babies?
110                                                                  If blacks support school choice and mandatory sentencing for criminals why don't they vote Republican?
114                               I am gay boy and I love my cousin (boy). He is sexy, but I dont know what to do. He is hot, and I want to see his di**. What should I do?
115                                                                                                                                    Which races have the smallest penis?
119                                                                                                                                       Wh

In [4]:
# #显示所有列
# pd.set_option('display.max_columns', None)
# #显示所有行
# pd.set_option('display.max_rows', None)
# #设置value的显示长度为100，默认为50
# pd.set_option('max_colwidth', 400)

# # print the longest sentence
# sentence_len = train_df["question_text"].apply(lambda x: len(str(x).split()))
# # add a col
# train_df["sentence_len"] = sentence_len
# print(train_df.head(5))
# longest_len = np.max(sentence_len)
# print(longest_len)
# longest_sentence = train_df.loc[train_df["sentence_len"] >= longest_len-70]
# # pd.set_option("max_colwidth", 10*int(longest_len))
# print(len(longest_sentence))
# print(longest_sentence)

In [5]:
import re

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# clean data
def clean_data(sentence):
    # 1. lower
    x = str(sentence)
    x = x.lower()
    
    # 2. punct data
    puncts = [
        ',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
        '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
        '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
        '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '₹', 'π', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    
    # 3. number data
    x = re.sub('[0-9]{5,}', ' ##### ', x)
    x = re.sub('[0-9]{4}', ' #### ', x)
    x = re.sub('[0-9]{3}', ' ### ', x)
    x = re.sub('[0-9]{2}', ' ## ', x)
    
    # 4. mispell data
    mispell_dict = {
        "aren't" : "are not",
        "can't" : "cannot",
        "couldn't" : "could not",
        "didn't" : "did not",
        "doesn't" : "does not",
        "don't" : "do not",
        "hadn't" : "had not",
        "hasn't" : "has not",
        "haven't" : "have not",
        "he'd" : "he would",
        "he'll" : "he will",
        "he's" : "he is",
        "i'd" : "I would",
        "i'd" : "I had",
        "i'll" : "I will",
        "i'm" : "I am",
        "isn't" : "is not",
        "it's" : "it is",
        "it'll":"it will",
        "i've" : "I have",
        "let's" : "let us",
        "mightn't" : "might not",
        "mustn't" : "must not",
        "shan't" : "shall not",
        "she'd" : "she would",
        "she'll" : "she will",
        "she's" : "she is",
        "shouldn't" : "should not",
        "that's" : "that is",
        "there's" : "there is",
        "they'd" : "they would",
        "they'll" : "they will",
        "they're" : "they are",
        "they've" : "they have",
        "we'd" : "we would",
        "we're" : "we are",
        "weren't" : "were not",
        "we've" : "we have",
        "what'll" : "what will",
        "what're" : "what are",
        "what's" : "what is",
        "what've" : "what have",
        "where's" : "where is",
        "who'd" : "who would",
        "who'll" : "who will",
        "who're" : "who are",
        "who's" : "who is",
        "who've" : "who have",
        "won't" : "will not",
        "wouldn't" : "would not",
        "you'd" : "you would",
        "you'll" : "you will",
        "you're" : "you are",
        "you've" : "you have",
        "'re": " are",
        "wasn't": "was not",
        "we'll":" will",
        "didn't": "did not",
        "tryin'":"trying",

        'colour':'color',
        'centre':'center',
        'didnt':'did not',
        'doesnt':'does not',
        'isnt':'is not',
        'shouldnt':'should not',
        'favourite':'favorite',
        'travelling':'traveling',
        'counselling':'counseling',
        'theatre':'theater',
        'cancelled':'canceled',
        'labour':'labor',
        'organisation':'organization',
        'wwii':'world war 2',
        'citicise':'criticize',
        'instagram': 'social medium',
        'whatsapp': 'social medium',
        'snapchat': 'social medium'
    }
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    def replace(match):
        return mispell_dict[match.group(0)]
    x = mispell_re.sub(replace, x)
    
    
    # 5. stop words
    tmp = ""
    for word in x.split():
        if word not in list(STOP_WORDS):
            tmp = tmp + word + " "
    x = tmp
    
    return x

# longest_sentence["question_text"] = longest_sentence["question_text"].apply(lambda x: clean_data(x))
# print(longest_sentence["question_text"])
# sentence_len = longest_sentence["question_text"].apply(lambda x: len(str(x).split()))
# print(sentence_len)

In [6]:
# Loading embedding
from gensim.models import KeyedVectors

GLOVE_PATH = os.path.join(DATA_PATH, "embeddings", "glove.840B.300d", "glove.840B.300d.txt")
glove_mean = -0.005838499
glove_std = 0.48782197
GOOGLENEWS_PATH = os.path.join(DATA_PATH, "embeddings", "GoogleNews-vectors-negative300", "GoogleNews-vectors-negative300.bin")
PARAGRAM_PATH =  os.path.join(DATA_PATH, "embeddings", "paragram_300_sl999", "paragram_300_sl999.txt")
paragram_mean = -0.0053247944
paragram_std = 0.49346468
WIKI_NEWS = os.path.join(DATA_PATH, "embeddings", "wiki-news-300d-1M", "wiki-news-300d-1M.vec")

def load_embedding(file):
    def get_coefs(word, *arr): 
        return word, np.asarray(arr, dtype='float32')
    if file == GOOGLENEWS_PATH:
        embeddings_index = KeyedVectors.load_word2vec_format(file, binary=True)
    elif file == os.path.join(DATA_PATH, "embeddings", "wiki-news-300d-1M", "wiki-news-300d-1M.vec"):
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index


In [7]:
from keras.preprocessing.text import Tokenizer

vocab_size = 120000

def build_vocab(sentences, verbose=True):
    # Tokenize the sentences
    tokenizer = Tokenizer(
        num_words=vocab_size,
        filters = '!"#$%()*+,-./:;<=>?@[\]^_`{|}~ '
    )
    tokenizer.fit_on_texts(sentences)
    vocab = tokenizer.word_counts
    print("word_index_len : ", len(vocab))
#     """
#     :param sentences: list of list of words
#     :return: dictionary of words and their count
#     """
#     vocab = {}
#     for sentence in tqdm(sentences, disable = (not verbose)):
#         for word in sentence.split():
#             try:
#                 vocab[word] += 1
#             except KeyError:
#                 vocab[word] = 1
#     print("word_index_len : ", len(vocab))
    return vocab

Using TensorFlow backend.


In [8]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [9]:
print("Begin load glove embedding ...")
embeddings_index = load_embedding(GLOVE_PATH)
print("Loading embedding completed!")

Begin load glove embedding ...
Loading embedding completed!


In [10]:
print("Begin clean data ...")
clean_sentences = train_df["question_text"].progress_apply(
    lambda x: clean_data(x))
print("Clean data completed!")

  0%|          | 429/1306122 [00:00<05:04, 4288.83it/s]

Begin clean data ...


100%|██████████| 1306122/1306122 [01:32<00:00, 14143.47it/s]

Clean data completed!


In [11]:
print("Begin build_vocab ...")
clean_vocab = build_vocab(clean_sentences)
print("Build vocab completed!")

Begin build_vocab ...
word_index_len :  185187
Build vocab completed!


In [12]:
clean_oov = check_coverage(clean_vocab, embeddings_index)
print(clean_oov[:20])

100%|██████████| 185187/185187 [00:00<00:00, 1201517.46it/s]

Found embeddings for 63.40% of vocab
Found embeddings for  98.56% of all text
[('quorans', 858), ('brexit', 524), ('cryptocurrencies', 499), ('redmi', 384), ('kvpy', 356), ('paytm', 356), ('iiser', 346), ('ethereum', 334), ('iisc', 278), ('jinping', 211), ('viteee', 186), ('iocl', 179), ('nmims', 163), ('upes', 157), ('rohingya', 157), ('fortnite', 156), ('coinbase', 149), ('nsit', 147), ('cpec', 146), ('iitians', 143)]


In [13]:
del embeddings_index
gc.collect()
print("Begin load paragram embedding ...")
embeddings_index = load_embedding(PARAGRAM_PATH)
print("Loading embedding completed!")

Begin load paragram embedding ...
Loading embedding completed!


In [14]:
clean_oov = check_coverage(clean_vocab, embeddings_index)
print(clean_oov[:20])

100%|██████████| 185187/185187 [00:00<00:00, 944253.40it/s] 

Found embeddings for 74.33% of vocab
Found embeddings for  99.16% of all text
[('quorans', 858), ('brexit', 524), ('cryptocurrencies', 499), ('redmi', 384), ('coinbase', 149), ('oneplus', 139), ('uceed', 124), ('demonetisation', 115), ('bhakts', 115), ('upwork', 111), ('machedo', 108), ('gdpr', 107), ('adityanath', 106), ('boruto', 102), ('bnbr', 100), ('alshamsi', 92), ('dceu', 90), ('litecoin', 87), ('iiest', 86), ('unacademy', 86)]


In [16]:
del embeddings_index
gc.collect()
print("Begin load paragram embedding ...")
embeddings_index = load_embedding(WIKI_NEWS)
print("Loading embedding completed!")

Begin load paragram embedding ...
Loading embedding completed!


In [17]:
clean_oov = check_coverage(clean_vocab, embeddings_index)
print(clean_oov[:20])

100%|██████████| 185187/185187 [00:00<00:00, 1280516.50it/s]

Found embeddings for 48.77% of vocab
Found embeddings for  97.19% of all text
[('upsc', 2179), ('aiims', 1038), ('cgl', 934), ('quorans', 858), ('jio', 770), ('manipal', 742), ('icse', 659), ('bitsat', 564), ('iiit', 564), ('cgpa', 563), ('ielts', 554), ('mtech', 504), ('ncert', 489), ('clat', 448), ('isro', 445), ('pilani', 390), ('ibps', 389), ('bhu', 386), ('redmi', 384), ('h1b', 380)]
